El servicio de venta de autos usados Rusty Bargain está desarrollando una aplicación para atraer nuevos clientes. Gracias a esa app, puedes averiguar rápidamente el valor de mercado de tu coche. Tienes acceso al historial: especificaciones técnicas, versiones de equipamiento y precios. Tienes que crear un modelo que determine el valor de mercado.
A Rusty Bargain le interesa:
- la calidad de la predicción;
- la velocidad de la predicción;
- el tiempo requerido para el entrenamiento

## Preparación de datos

In [22]:
import pandas as pd #Manipular Datasets
import numpy as np #Manipular Datos y operaciones
import seaborn as sb #Pairplot
import matplotlib #Graficas
#Machine Learning
from sklearn.metrics import mean_squared_error as mse
from sklearn.linear_model import LinearRegression 
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.preprocessing import StandardScaler, OneHotEncoder, OrdinalEncoder
from sklearn.ensemble import RandomForestRegressor
#Potenciación
import lightgbm as lgb
from catboost import CatBoostRegressor as CBR
#Calculos temporales
import time


cars = pd.read_csv('/datasets/car_data.csv')


In [2]:
cars.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 354369 entries, 0 to 354368
Data columns (total 16 columns):
 #   Column             Non-Null Count   Dtype 
---  ------             --------------   ----- 
 0   DateCrawled        354369 non-null  object
 1   Price              354369 non-null  int64 
 2   VehicleType        316879 non-null  object
 3   RegistrationYear   354369 non-null  int64 
 4   Gearbox            334536 non-null  object
 5   Power              354369 non-null  int64 
 6   Model              334664 non-null  object
 7   Mileage            354369 non-null  int64 
 8   RegistrationMonth  354369 non-null  int64 
 9   FuelType           321474 non-null  object
 10  Brand              354369 non-null  object
 11  NotRepaired        283215 non-null  object
 12  DateCreated        354369 non-null  object
 13  NumberOfPictures   354369 non-null  int64 
 14  PostalCode         354369 non-null  int64 
 15  LastSeen           354369 non-null  object
dtypes: int64(7), object(

In [3]:
cars.sample(5)

,DateCrawled,Price,VehicleType,RegistrationYear,Gearbox,Power,Model,Mileage,RegistrationMonth,FuelType,Brand,NotRepaired,DateCreated,NumberOfPictures,PostalCode,LastSeen
76303,01/04/2016 13:45,1300,small,2003,manual,60,2_reihe,150000,8,petrol,peugeot,no,01/04/2016 00:00,0,21031,07/04/2016 10:17
31108,04/04/2016 18:56,5450,small,2008,NaN,45,fortwo,100000,7,gasoline,smart,no,04/04/2016 00:00,0,49090,06/04/2016 21:16
226533,26/03/2016 23:55,7500,wagon,2005,auto,272,5er,150000,6,gasoline,bmw,no,26/03/2016 00:00,0,92637,05/04/2016 01:17
13769,05/04/2016 09:56,360,small,1997,manual,75,fiesta,150000,9,petrol,ford,NaN,05/04/2016 00:00,0,45892,07/04/2016 13:17
349469,15/03/2016 21:37,0,sedan,2005,auto,218,3er,90000,5,petrol,bmw,no,15/03/2016 00:00,0,10409,24/03/2016 16:17


In [4]:
#cars['Model'].unique()
#othermodel= cars['Model']=='other'
#cars['Model'].isna().value_counts()

In [5]:
cars['VehicleType'].fillna('unknown', inplace=True) #Creamos un valor para los desconocidos
cars['Gearbox'].fillna('manual', inplace=True) #Se llena con el valor más repetido
cars['Model'].fillna('unknown', inplace=True) #Creamos un valor para los desconocidos
cars['FuelType'].replace('petrol','gasoline', inplace=True) #Reemplazamos este valor porque son sinonimos
cars['FuelType'].fillna('unknown', inplace=True) #Creamos un valor para los desconocidos
cars['NotRepaired'].fillna('unknown', inplace=True) #Creamos un valor para los desconocidos

In [6]:
# Dropeamos algunas caracteristicas que no son relevantes para el analisis
cars.drop(['DateCrawled','RegistrationMonth','DateCreated','LastSeen'], axis= 1, inplace= True)
#El mes de registro se dropea porque generalmente lo relevante es el año del vehiculo

In [7]:
cars.info() #Confirmamos df sin na

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 354369 entries, 0 to 354368
Data columns (total 12 columns):
 #   Column            Non-Null Count   Dtype 
---  ------            --------------   ----- 
 0   Price             354369 non-null  int64 
 1   VehicleType       354369 non-null  object
 2   RegistrationYear  354369 non-null  int64 
 3   Gearbox           354369 non-null  object
 4   Power             354369 non-null  int64 
 5   Model             354369 non-null  object
 6   Mileage           354369 non-null  int64 
 7   FuelType          354369 non-null  object
 8   Brand             354369 non-null  object
 9   NotRepaired       354369 non-null  object
 10  NumberOfPictures  354369 non-null  int64 
 11  PostalCode        354369 non-null  int64 
dtypes: int64(6), object(6)
memory usage: 32.4+ MB


In [8]:
cars.describe()

,Price,RegistrationYear,Power,Mileage,NumberOfPictures,PostalCode
count,354369.000000,354369.000000,354369.000000,354369.000000,354369.0,354369.000000
mean,4416.656776,2004.234448,110.094337,128211.172535,0.0,50508.689087
std,4514.158514,90.227958,189.850405,37905.341530,0.0,25783.096248
min,0.000000,1000.000000,0.000000,5000.000000,0.0,1067.000000
25%,1050.000000,1999.000000,69.000000,125000.000000,0.0,30165.000000
50%,2700.000000,2003.000000,105.000000,150000.000000,0.0,49413.000000
75%,6400.000000,2008.000000,143.000000,150000.000000,0.0,71083.000000
max,20000.000000,9999.000000,20000.000000,150000.000000,0.0,99998.000000


In [9]:
cars_num= cars[['Price','RegistrationYear','Power','Mileage',
    'NumberOfPictures','PostalCode']]
data_sample= cars_num.sample(15000)

In [10]:
#g = sb.pairplot(data_sample, kind='hist')
#g # Buscamos correlaciones visuales

<div class="alert alert-block alert-success">
<b>Comentario del revisor (1ra Iteracion)</b> <a class=“tocSkip”></a>

Muy buen trabajo con el tratamiento y análisis inicial de los datos, siempre en un proyecto lo importante es primero entender los datos con los que se trabajará antes de pasar al modelado
</div>

## Entrenamiento del modelo 

In [17]:
#Asignamos target
features= cars.drop('Price', axis= 1)
target= cars['Price']

for col in cat_cols:
    features[col]= features[col].astype('category') #Definimos explicitamente para que LightGBM detecte datos categoricos
    
#Separamos datos para entrenamiento
features_train, features_temp, target_train, target_temp = train_test_split(features, target, test_size=.4, random_state= 123)
features_test, features_valid, target_test, target_valid = train_test_split(features_temp, target_temp, test_size=.5, random_state= 123)

#Identificamos columnas numericas y categoricas
cat_cols= ['VehicleType','Gearbox','Model','FuelType','Brand','NotRepaired']
num_cols= features_train.select_dtypes(include= np.number).columns.tolist()
num_cols= [col for col in num_cols if col not in cat_cols] #Sacamos las columnas numericas que no están en cat_cols

state= np.random.RandomState(12345) # definimos estado aleatorio

In [12]:
#Codificacón y escalamiento para linear regresion y arboles

encoder= OneHotEncoder(sparse= False, handle_unknown= 'ignore') #Para codificar caracteristicas nominales
scaler= StandardScaler() #Escalador

feat_preohe = encoder.fit_transform(features_train[cat_cols]) #fit transform para features categoricas
ohe_col_names = encoder.get_feature_names(cat_cols)
feat_ohe= pd.DataFrame(feat_preohe, columns= ohe_col_names) #Recuperar nombres de columnas

feat_presc= scaler.fit_transform(features_train[num_cols]) #Para escalar numericas
feat_scaled= pd.DataFrame(feat_presc, columns= num_cols) 

feat_train_processed= pd.concat([feat_scaled, feat_ohe], axis= 1) #Incluimos las columnas numericas
feat_train_processed.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 212621 entries, 0 to 212620
Columns: 317 entries, RegistrationYear to NotRepaired_yes
dtypes: float64(317)
memory usage: 514.2 MB


In [13]:
#Procesamos el ds para test
test_encoded= encoder.transform(features_test[cat_cols])
test_encoded= pd.DataFrame(test_encoded, columns= ohe_col_names)

test_scaled= scaler.transform(features_test[num_cols])
test_scaled= pd.DataFrame(test_scaled, columns= num_cols)

features_test_processed= pd.concat([test_scaled, test_encoded], axis= 1) #dataset de test preprocesado

<div class="alert alert-block alert-danger">
<b>Comentario del revisor (1ra Iteracion)</b> <a class=“tocSkip”></a>

Muy buen avance con el proyecto Ricardo! Tienes los entrenamientos de manera adecuada. Sin embargo, si revisamos la descripción del proyecto dice que a la empresa le interesa el tiempo de entrenamiento y la de velocidad de predicción que a fin de cuentas también se puede medir con el tiempo que tarde, por lo que deberías aplicar la librería `time` tanto para cuando haces `.fit()` cómo para cuando haces `.predict()`, es decir, deberías tener dos resultados en cuanto a tiempo para los modelos.

Y no olvides que siempre al final de tus proyectos debes tener una sección para las conclusioens del mismo.
   
</div>

<div class="alert alert-block alert-info">
<b>Hola Patricio, ya incluí la medición de los tiempos de entrenamiento y predicción para cada modelo.</b> <a class=“tocSkip”></a>
</div>

In [24]:
#Prueba de Regresion Lineal
lrf_stime= time.time() #inicio de medicion de tiempo
lmodel_fitted= LinearRegression().fit(feat_train_processed, target_train)
lrf_etime= time.time() #fin de medición de tiempo

lrp_stime= time.time()
lpredictions= lmodel_fitted.predict(features_test_processed)
lrp_etime= time.time()

print('RECM de Regresión Lineal:', mse(target_test, lpredictions)**.5)

lmodel= LinearRegression()
lscores= cross_val_score(lmodel, feat_train_processed, target_train, cv= 5, scoring='neg_mean_squared_error')

print('Validación cruzada:')
print('ECMs negativos:', lscores)
print('RECM promedio:', (-lscores.mean())**.5)
print('Desviación estadar ECM:', lscores.std()) 
print('Tiempo de procesamiento fit en segundos', lrf_etime-lrf_stime)
print('Tiempo de procesamiento predict en segundos', lrp_etime-lrp_stime)


RECM de Regresión Lineal: 3265.016530134586
Validación cruzada:
ECMs negativos: [-2.77279416e+21 -1.06477592e+07 -1.06522368e+07 -1.07146449e+07
 -1.08626839e+07]
RECM promedio: 23549072848.37858
Desviación estadar ECM: 1.1091176640364608e+21
Tiempo de procesamiento fit en segundos 8.069927453994751
Tiempo de procesamiento predict en segundos 0.09133720397949219


In [ ]:
#Prueba de Bosque
ff_stime= time.time()
fmodel_fitted= RandomForestRegressor(random_state= state, n_estimators= 60, max_depth= 10).fit(rfeat_train_processed, target_train)
ff_etime= time.time()

fp_stime= time.time()
fpredictions= fmodel_fitted.predict(features_test_processed)
fp_etime= time.time()

print('RECM de Bosque Aleatorio:', mse(target_test, fpredictions)**.5)

fmodel= RandomForestRegressor(random_state= state, n_estimators= 60, max_depth= 10)
fscores= cross_val_score(fmodel, feat_train_processed, target_train, cv= 5, scoring='neg_mean_squared_error') # Validamos el bosque

print('ECMs negativos:', fscores)
print('RECM promedio:', (-fscores.mean())**.5)
print('Desviación estadar ECM:', fscores.std())
print('Tiempo de procesamiento fit en segundos', ff_etime-ff_stime)
print('Tiempo de procesamiento predict en segundos', fp_etime-fp_stime)

RECM de Bosque Aleatorio: 2025.0245749710007
ECMs negativos: [-4132864.05150974 -4177466.81333694 -4126128.77945163 -4167618.95895638
 -4239794.8703681 ]
RECM promedio: 2041.7577463363662
Desviación estadar ECM: 40568.466535341024
Tiempo de procesamiento fit en segundos 104.14913320541382
Tiempo de procesamiento predict en segundos 0.3114185333251953


In [ ]:
#Prueba de potenciación con LightGBM
gbm = lgb.LGBMRegressor(num_leaves= 31, learning_rate= .05, n_estimators=20, random_state= state)

lgf_stime= time.time()
gbm.fit(features_train, target_train, eval_set=[(features_test, target_test)],eval_metric='l2', callbacks=[lgb.early_stopping(5)])
lgf_etime= time.time()

lgp_stime= time.time()
gbpred = gbm.predict(features_test, num_iteration= gbm.best_iteration_)
lgp_etime= time.time()

rmse_test = mse(target_test, gbpred) ** 0.5
print(f'EL RECM de potenciación con LightGBM es: {rmse_test}')
print('Tiempo de procesamiento fit en segundos', lgf_etime-lgf_stime)
print('Tiempo de procesamiento predict en segundos', lgp_etime-lgp_stime)


/.venv/lib/python3.9/site-packages/lightgbm/basic.py:1780: UserWarning: Overriding the parameters from Reference Dataset.
  _log_warning('Overriding the parameters from Reference Dataset.')
/.venv/lib/python3.9/site-packages/lightgbm/basic.py:1513: UserWarning: categorical_column in param dict is overridden.
  _log_warning(f'{cat_alias} in param dict is overridden.')


Training until validation scores don't improve for 5 rounds
Did not meet early stopping. Best iteration is:
[20]	valid_0's l2: 6.96309e+06
EL RECM de potenciación con LightGBM es: 2638.7659714463866
Tiempo de procesamiento fit en segundos 0.6952037811279297
Tiempo de procesamiento predict en segundos 0.0992889404296875


In [28]:
#Prueba con CatBoost
gbm2= CBR(random_seed= 12345, learning_rate= .05, iterations= 150, depth= 10, verbose= 25)

cbf_stime= time.time()
gbm2.fit(features_train, target_train,cat_features= cat_cols)
cbf_etime= time.time()

cbp_stime= time.time()
cb_pred= gbm2.predict(features_test)
cbp_etime= time.time()

print('RECM de CatBoost:', mse(target_test, cb_pred)**.5)
print('Tiempo de procesamiento fit en segundos', cbf_etime-cbf_stime)
print('Tiempo de procesamiento predict en segundos', cbp_etime-cbp_stime)

0:	learn: 4366.4331535	total: 72.9ms	remaining: 10.9s
25:	learn: 2487.4937859	total: 1.23s	remaining: 5.88s
50:	learn: 2060.4443385	total: 2.36s	remaining: 4.59s
75:	learn: 1944.6767712	total: 3.52s	remaining: 3.43s
100:	learn: 1898.0312949	total: 4.76s	remaining: 2.31s
125:	learn: 1865.8109330	total: 6.12s	remaining: 1.17s
149:	learn: 1835.4815076	total: 7.43s	remaining: 0us
RECM de CatBoost: 1860.2177571116863
Tiempo de procesamiento fit en segundos 7.723387718200684
Tiempo de procesamiento predict en segundos 0.04309701919555664


## Análisis del modelo

Los diferentes modeos tuvieron sus particularidades. La regresión lineal tuvo el mayor error y la mayor variación pero era el modelo base para comparar los demás.
Tanto la regresión lineal como el bosque se tuvieron que procesar para poder implementarse, y el bosque tuvo una mejor puntuación o menor error que la potenciación con LightGBM. Aunque fue el modelo que más tardo y por mucho, en entrenarse y predecir.
Pero el modelo más rápido, tuvimos el de LightGBM, pero el menor error de todos fue CatBoost, por lo que es el modelo a elegir en este caso, ya que aun que es un poco más lento que LightGBM, La diferencia en tiempo no es tanta y el error es bastante más bajo.

<div class="alert alert-block alert-success">
<b>Comentario del revisor (2da Iteracion)</b> <a class=“tocSkip”></a>

Muy bien, ahora se evaluaron correctamente las métricas solicitadas, en cuanto a las métricas de desempeño de las predicciones cómo en el tiempo que toma cada una de sus etapas. El medir el tiempo es importante ya que cuando pones los modelos en producción para que puedan ser usados se suele realizar por medio de API donde se prioriza más el tiempo de predicción que la precisión cómo tal. Lo ideal es buscar siempre el balance entre buen desempeño de predicciones y de tiempo de predicción para tu modelo, sobre todo cuando va a ser usado en tiempo real.
    
Saludos!
</div>

# Lista de control

Escribe 'x' para verificar. Luego presiona Shift+Enter

- [x]  Jupyter Notebook está abierto
- [x]  El código no tiene errores- [ ]  Las celdas con el código han sido colocadas en orden de ejecución- [ ]  Los datos han sido descargados y preparados- [ ]  Los modelos han sido entrenados
- [x]  Se realizó el análisis de velocidad y calidad de los modelos